# NeuroNerd Fine-Tuning with Unsloth 🦥

This notebook fine-tunes **Llama 3.1 8B** on your custom dataset using Unsloth (2x faster, 60% less memory).

**Hardware:** Requires GPU (Google Colab T4 is perfect).

In [ ]:
# 1. Install Unsloth and Dependencies
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

In [ ]:
# 2. Setup Model & Tokenizer
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# Add LoRA adapters so we only train 1-10% of parameters
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [ ]:
# 3. Load Your Dataset
from datasets import load_dataset

# OPTION A: If you uploaded files to Colab directly
dataset_files = {"train": "train.jsonl", "test": "test.jsonl"}

# OPTION B: Clone from GitHub (Uncomment if needed)
# !git clone https://github.com/YOUR_USERNAME/neuronerd.git
# dataset_files = {"train": "neuronerd/output/final/train.jsonl", "test": "neuronerd/output/final/test.jsonl"}

dataset = load_dataset("json", data_files=dataset_files)

# Format function for Alpaca style
alpaca_prompt = """Below is an instruction that describes a task, backed by an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

dataset = dataset.map(formatting_prompts_func, batched = True)

In [ ]:
# 4. Train the Model
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset["train"],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60, # Increase to 300+ for full training (about 1 epoch)
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

trainer_stats = trainer.train()

In [ ]:
# 5. Save Model to GGUF (for local use in Ollama)
# Save to 8bit Q8_0
model.save_pretrained_gguf("neuronerd_model", tokenizer, quantization_method = "q8_0")

# Push to Hub (Optional)
# model.push_to_hub_gguf("your_username/neuronerd", tokenizer, quantization_method = "q8_0", token = "hf_...")